In [1]:
import os
import requests

In [2]:
import utils

utils.load_env()

In [3]:
from langchain_community.tools import GooglePlacesTool

places = GooglePlacesTool()
places.run("fasion islands")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `GooglePlacesTool` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GooglePlacesTool`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `GooglePlacesAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GooglePlacesAPIWrapper`.
  w

'1. Fashion Island\nAddress: 587-589/7-9, Ram Inthra Rd, Khwaeng Khan Na Yao, Khet Khan Na Yao, Krung Thep Maha Nakhon 10230, Thailand\nGoogle place ID: ChIJd57HEQRjHTERG_xSTcdQoYs\nPhone: 02 947 5000\nWebsite: http://www.fashionisland.co.th/\n\n\n2. Food Island-Fashion Island\nAddress: 5/5-6 หมู่ 5 Fashion Island, Ram Inthra Rd, Khwaeng Khan Na Yao, บางเขน Krung Thep Maha Nakhon 10220, Thailand\nGoogle place ID: ChIJxVlvKA9jHTERAaDXThEk0sU\nPhone: Unknown\nWebsite: https://www.fashionisland.co.th/\n\n\n3. Grand Station @ Fashion Island\nAddress: Fashion Island Shopping Mall คันนายาว Khet Khan Na Yao, Krung Thep Maha Nakhon 10230, Thailand\nGoogle place ID: ChIJgYJ0xNxjHTERqh7EMAXGflk\nPhone: 02 947 5000\nWebsite: https://www.fashionisland.co.th/about-us/grandstation/\n\n\n4. Central Department Store Fashion Island\nAddress: 591 Ram Inthra Rd, Khwaeng Khan Na Yao, Khet Khan Na Yao, Krung Thep Maha Nakhon 10230, Thailand\nGoogle place ID: ChIJU51oO09jHTERkAkthlfrvMQ\nPhone: 02 947 5320\

In [4]:
#find places id
#near by search

In [5]:
def find_place_from_text(input_text, location=None, radius=2000):
    # Retrieve the API key from environment variables
    api_key = os.getenv('GPLACES_API_KEY')

    if not api_key:
        raise ValueError("API key not found. Please set the GOOGLE_MAPS_API_KEY environment variable.")

    # Define the endpoint URL
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

    # Define the parameters for the request
    params = {
        'fields': 'formatted_address,name,rating,opening_hours,geometry',
        'input': input_text,
        'inputtype': 'textquery',
        'key': api_key
    }
    
    params['locationbias'] = f'circle:{radius}@{location}' if location is not None and radius is not None else None

    # Make the request to the Google Maps API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        response.raise_for_status()  # Raise an exception for HTTP errors

# Example usage:
location = "15.87,100.9925"  # Latitude and Longitude
input_text = "มาบุญครอง"
result = find_place_from_text(input_text, location)

print(result)

{'candidates': [{'formatted_address': '444 ถ. พญาไท แขวงวังใหม่ เขตปทุมวัน กรุงเทพมหานคร 10330 ไทย', 'geometry': {'location': {'lat': 13.744677, 'lng': 100.5295593}, 'viewport': {'northeast': {'lat': 13.74598387989272, 'lng': 100.5309299798927}, 'southwest': {'lat': 13.74328422010728, 'lng': 100.5282303201073}}}, 'name': 'เอ็ม บี เค เซ็นเตอร์', 'opening_hours': {'open_now': True}, 'rating': 4.3}], 'status': 'OK'}


In [6]:
def find_location(input_text:str, location:str=None, radius=10000):
    # Call the find_place_from_text function to get the location data
    data = find_place_from_text(input_text, location, radius)

    # Extract the latitude and longitude from the response
    candidates = data.get('candidates', [])
    if len(candidates)==0:
        raise ValueError("No location found.")

    # Assuming we're taking the first candidate
    geometry = candidates[0].get('geometry', {})
    location = geometry.get('location', {})

    latitude = location.get('lat')
    longitude = location.get('lng')

    if latitude is None or longitude is None:
        raise ValueError("Latitude or Longitude not found in the response.")

    # Return the latitude and longitude as a formatted string
    return f"{latitude},{longitude}"


location = "15.87,100.9925"  # Latitude and Longitude
input_text = "มาบุญครอง"
coordinates = find_location(input_text, location)
print(f"Coordinates: {coordinates}")

Coordinates: 13.744677,100.5295593


In [17]:
def nearby_search(keyword:str, location:str, radius=2000, place_type=None):
    # Retrieve the API key from environment variables
    api_key = os.getenv('GPLACES_API_KEY')

    if not api_key:
        raise ValueError("API key not found. Please set the GOOGLE_MAPS_API_KEY environment variable.")

    # Define the endpoint URL
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Set up the parameters for the request
    params = {
        'keyword': keyword,
        'location': location,
        'radius': radius,
        'type': place_type,
        'key': api_key,
        "rankPreference": "DISTANCE"
    }

    # Send the GET request to the Google Maps API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Error with request: {response.status_code}, {response.text}")

    # Parse the JSON response
    data = response.json()
    results = data['results']

    # search into next page
    while data.get('next_page_token', False):
        params = {'next_page_token': data['next_page_token']}
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            raise Exception(f"Error with request: {response.status_code}, {response.text}")
        
        data = response.json()
        
        results.append(data['results'])
        

    # Return the response data
    return results


# Example usage
location = "13.744677,100.5295593"  # Latitude and Longitude
keyword = "ร้านอาหาร"
search_results = nearby_search(keyword, location)
len(search_results)

21